## Machine Learning

In [19]:

# can also do a classifer that tries to predict the strategy used given the other columns (part characteristics)

# next step: can you make a multi-class classifier + regression model? 

# classify the strategy -> classify the strategy that yields the optimal util, GIVEN the part characteristics

In [20]:
# CORE
import pandas as pd
import numpy as np  # Numpy for numerical computations and array operations
import pandas as pd  # Pandas for data manipulation and analysis

# MACHINE LEARNING & STATISTICS 
import scipy.stats as stats  # SciPy for scientific computing and technical computing, including statistics
import sklearn as sk # Scikit-learn for machine learning and predictive modeling
import pycaret # PyCaret for low-code machine learning and data science automation
from pycaret.regression import * # PyCaret's regression module

# VISUALIZATION
import matplotlib.pyplot as plt  # Matplotlib for creating static, animated, and interactive visualizations
import seaborn as sns  # Seaborn for statistical data visualization built on top of Matplotlib
import plotly.express as px  # Plotly Express for creating interactive plots and charts
import seaborn as sns

In [21]:
# read in the ml_data.csv file
ml_df = pd.read_csv("../CEIP_csv/ml_data.csv")

In [22]:
util = 'calcUtil' # define the target

### XGBoost

In [23]:
# implement XGBoost on this model
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

In [24]:
# Define the features and target variables
# Assuming that 'target' is the name of the column that we want to predict
ml_df.drop('Unnamed: 0', axis=1, inplace=True) # remove unnamed column 

In [30]:
# PREPROCESSING FOR XGBOOST 
# Replace positive infinity with the maximum non-infinity number in the dataset
ml_df.replace([np.inf], np.finfo('float32').max, inplace=True)

# Replace negative infinity with the minimum non-infinity number in the dataset
ml_df.replace([-np.inf], np.finfo('float32').min, inplace=True)

# Check if the dataframe still contains NaN and Inf values and replace them with the mean of the column
for col in ml_df.columns:
    if ml_df[col].isnull().any():
        ml_df[col].fillna(ml_df[col].mean(), inplace=True)

    if np.isinf(ml_df[col]).any():
        ml_df[col].replace([np.inf, -np.inf], ml_df[col].mean(), inplace=True)
        
#  Replace inf and -inf with np.nan, then drop all rows with np.nan
ml_df = ml_df.replace([np.inf, -np.inf], np.nan)
ml_df = ml_df.dropna()
        
ml_df = ml_df.dropna()

In [31]:
# remove outliers 
# Compute the IQR
Q1 = ml_df.quantile(0.25)
Q3 = ml_df.quantile(0.75)
IQR = Q3 - Q1

# Determine the outlier cutoff
lower_bound = Q1 - 2 * IQR
upper_bound = Q3 + 2 * IQR

# Create a mask for values within the IQR
mask = (ml_df >= lower_bound) & (ml_df <= upper_bound)

# Apply the mask to remove outliers
ml_df_no_outliers = ml_df[mask].dropna()

In [32]:
# Split the data into training and testing sets
X = ml_df.drop(util, axis=1)
y = ml_df[util]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [35]:
# convert everything to the float data type 
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
y_train = y_train.astype('float32')
y_test = y_test.astype('float32')

# drop all NaN values
X_train = X_train.dropna()
X_test = X_test.dropna()
y_train = y_train.dropna()
y_test = y_test.dropna()

In [34]:
# apply scaling 
from sklearn.preprocessing import StandardScaler

# Initialize the scaler
scaler = StandardScaler()

# Fit the scaler and transform the training data
X_train_scaled = scaler.fit_transform(X_train)

# Use the same scaler to transform the test data
X_test_scaled = scaler.transform(X_test)

ValueError: Input contains infinity or a value too large for dtype('float32').

In [ ]:
# Check for 'nan' or 'inf' in X_train_scaled and y_train
assert not np.any(np.isnan(X_train_scaled)), 'X_train_scaled contains NaN'
assert not np.any(np.isnan(y_train)), 'y_train contains NaN'
assert not np.any(np.isinf(X_train_scaled)), 'X_train_scaled contains Inf'
assert not np.any(np.isinf(y_train)), 'y_train contains Inf'

# Check for 'nan' or 'inf' in X_test_scaled and y_test
assert not np.any(np.isnan(X_test_scaled)), 'X_test_scaled contains NaN'
assert not np.any(np.isnan(y_test)), 'y_test contains NaN'
assert not np.any(np.isinf(X_test_scaled)), 'X_test_scaled contains Inf'
assert not np.any(np.isinf(y_test)), 'y_test contains Inf'

AssertionError: X_train_scaled contains NaN

In [ ]:
# Create an XGBoost model
# use the regression model to minimize mean squared error in predictions 

# set model parameters 
params = {
    'objective': 'reg:squarederror',
    'colsample_bytree': 0.5,
    'learning_rate': 0.1,
    'max_depth': 5,
    'alpha': 10,
    'n_estimators': 100,
    'subsample': 0.8,
    'min_child_weight': 1,
    'gamma': 0
}

model = xgb.XGBRegressor(**params)

# train the model 
model.fit(X_train_scaled, y_train)

XGBoostError: [19:23:55] /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-10.9-x86_64-cpython-38/xgboost/src/data/data.cc:1104: Check failed: valid: Input data contains `inf` or `nan`
Stack trace:
  [bt] (0) 1   libxgboost.dylib                    0x00000001354e4785 dmlc::LogMessageFatal::~LogMessageFatal() + 117
  [bt] (1) 2   libxgboost.dylib                    0x000000013556d4f0 unsigned long long xgboost::SparsePage::Push<xgboost::data::ArrayAdapterBatch>(xgboost::data::ArrayAdapterBatch const&, float, int) + 1280
  [bt] (2) 3   libxgboost.dylib                    0x00000001355ba71b xgboost::data::SimpleDMatrix::SimpleDMatrix<xgboost::data::ArrayAdapter>(xgboost::data::ArrayAdapter*, float, int) + 299
  [bt] (3) 4   libxgboost.dylib                    0x000000013556ba35 xgboost::DMatrix* xgboost::DMatrix::Create<xgboost::data::ArrayAdapter>(xgboost::data::ArrayAdapter*, float, int, std::__1::basic_string<char, std::__1::char_traits<char>, std::__1::allocator<char

In [ ]:
# Let's take the first 100,000 rows as an example
small_X_train = X_train[:100000]
small_y_train = y_train[:100000]

# Fit the model on the smaller subset
model.fit(small_X_train, small_y_train)

In [ ]:
# Make predictions
y_pred = model.predict(X_test)

# Evaluate the model
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
print("RMSE: %f" % (rmse))

### Basic models to try 
* Linear Regression - Use sci-kit learn to implement
* Logistic Regression
* Support Vector Machines
* Basic decision trees
* Naive Bayes

### XGBoost 
Tree ensemble model that can handle tabular, numerical, low-dimensional data very well. Fast and scalable, and can be hyperparameter tuned to optimize performance.

### ResNet-like architecture
Neural network architecture adapted for tabular data, allowing learning from shallow and deep features. Use PyTorch or TensorFlow to implement. Less interpretable. 

### Transformer 
Use Transformers with tabular data? 

### KNN - K Nearest Neighbors
Predicts the target variable based on the similarity of the features with the nearest neighbors in the training dataset. Computationally expensive, but can be used for regression and classification.

### Model Evaluation
* Cross-validation
* Classification Accuracy
* Confusion Matrix
* ROC-AUC Curve
  

### Model Fine-tuning
* Hyperparameter tuning
* Ensembling multiple models together - voting classifier, bagging, boosting, XGBoost 

### Feature Importance

**Univariate Feature Selection:** This method uses statistical tests like chi-squared tests or ANOVA to evaluate the relationship between each feature and the target variable independently. It ranks the features based on their significance.

In [ ]:
# from sklearn.feature_selection import SelectKBest, chi2, f_classif

# # For categorical target using Chi-squared test
# selector = SelectKBest(score_func=chi2, k=5)
# selector.fit_transform(X, y)

# # For continuous target using ANOVA F-value
# selector = SelectKBest(score_func=f_classif, k=5)
# selector.fit_transform(X, y)

# # Get feature importances
# scores = selector.scores_
# feature_importances = pd.DataFrame({'feature': X.columns, 'importance': scores})
# feature_importances = feature_importances.sort_values('importance', ascending=False)
# print(feature_importances)

In [ ]:
# other statistical testing 
# import scipy.stats as stats

# # Perform a t-test to compare the means of two groups
# t_stat, p_value = stats.ttest_ind(group1, group2)
# print("T-statistic:", t_stat, "P-value:", p_value)

# # Perform a chi-squared test to determine the association between two categorical variables
# chi_stat, p_value, dof, ex = stats.chi2_contingency(contingency_table)
# print("Chi-squared statistic:", chi_stat, "P-value:", p_value)